## Code fourni pour lire les binaires:

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time

def read_int(f):
    ba = bytearray(4)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.int32)
    return prm[0]

def read_double(f):
    ba = bytearray(8)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.double)
    return prm[0]

def read_double_tab(f, n):
    ba = bytearray(8*n)
    nr = f.readinto(ba)
    if nr != len(ba):
        return []
    else:
        prm = np.frombuffer(ba, dtype=np.double)
        return prm

def get_pics_from_file(filename):
    # Lecture du fichier d'infos + pics detectes (post-processing KeyFinder)
    #print("Ouverture du fichier de pics "+filename)
    f_pic = open(filename, "rb")
    info = dict()
    info["nb_pics"] = read_int(f_pic)
    #print("Nb pics par trame: " + str(info["nb_pics"]))
    info["freq_sampling_khz"] = read_double(f_pic)
    #print("Frequence d'echantillonnage: " + str(info["freq_sampling_khz"]) + " kHz")
    info["freq_trame_hz"] = read_double(f_pic)
    #print("Frequence trame: " + str(info["freq_trame_hz"]) + " Hz")
    info["freq_pic_khz"] = read_double(f_pic)
    #print("Frequence pic: " + str(info["freq_pic_khz"]) + " kHz")
    info["norm_fact"] = read_double(f_pic)
    #print("Facteur de normalisation: " + str(info["norm_fact"]))
    tab_pics = []
    pics = read_double_tab(f_pic, info["nb_pics"])
    nb_trames = 1
    while len(pics) > 0:
        nb_trames = nb_trames+1
        tab_pics.append(pics)
        pics = read_double_tab(f_pic, info["nb_pics"])
    #print("Nb trames: " + str(nb_trames))
    f_pic.close()
    return tab_pics, info

    """
    ######### Pics ############
    # NO KEY
    plt.figure(1)
    plt.subplot(211)
    plt.plot(range(1,info["nb_pics"]+1), np.mean(pics_nokey,axis=0), 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('no key')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    # PAD-0
    plt.subplot(212)
    plt.plot(range(1,info["nb_pics"]+1), pics_nokey[0], 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('PAD-0')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    #
    plt.show()
    """

# Notre Algorithme
* Séparer les trames en chunk ------- While == précédent + [un certain pas]
* Attention à shift+a par exemple --- Maybe in top best letters ?
* ctrl alt suppr au début ---- Histoire de pas oublier

### Bin to png

In [282]:
#for file in glob.glob("../data/pics_*.bin"):
 #   key = file.split("pics_")[1].replace(".bin","")
 #   values, info = get_pics_from_file(file)
 #   plt.imshow(values,aspect="auto",)
 #   plt.savefig("../data/PNGs/pics_" + key + ".png")

### Bibliothèques utilisées :

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
import glob

## Constitution du dataset

In [4]:
# Il faut enlever le bruit

X,Y = [],[]

pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")

letters = 0
nokey = np.mean(pics_nokey,axis=0)
for file in glob.glob("../data/pics_*.bin"):
    key = file.split("pics_")[1].replace(".bin","")
    if key == "LOGINMDP":
        continue
    values, info = get_pics_from_file(file)
    np.random.shuffle(values)
    values = values[:3000]
    values = np.subtract(values,nokey)
    mean = np.mean(values,axis=0)
    values = values * mean
    for frame in values:
        X.append(frame)
        Y.append(key)
    letters += 1
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

# Scaling des données
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### KNN Algorithm

In [285]:
import math
# Best value for N (internet said this)
N = math.floor(math.sqrt(len(Y_test)))
N = math.floor(math.sqrt(47))
if N % 2 == 0:
    N -= 1
print(N)

5


In [5]:
# KNN Clustering
classifier = KNeighborsClassifier(n_neighbors=40, p=2, metric='euclidean',n_jobs=-1)
classifier.fit(X_train,Y_train)
Y_pred = classifier.predict(X_test)
print(Y_pred)

#### K-cross validation ###
#from sklearn.model_selection import cross_val_score
#cv_scores = cross_val_score(classifier,X,Y, cv=10)
#print(cv_scores)
#print(np.mean(cv_scores))

### Evaluating model ###
print(Y_test)
print(accuracy_score(Y_test,Y_pred),"%")
#cm = confusion_matrix(Y_test,Y_pred)
# La diagonale c'est les bons guess et le reste les fautes
#print(cm)

['H' 'SPACE' 'SHIFT' ... '2' '4' 'H']
['U', 'SPACE', 'SHIFT', 'SPACE', 'B', 'E', 'L', 'SPACE', 'V', 'B', '4', 'L', 'W', 'SHIFT', 'R', 'Q', 'M', 'Z', '1', 'G', 'T', 'SUPPR', 'D', 'SHIFT', 'E', 'D', '6', '2', '1', 'H', 'SUPPR', 'R', '1', '3', 'J', 'H', 'G', 'I', '1', 'X', 'L', 'S', 'NOKEY', 'P', 'Q', 'A', 'W', 'H', 'SUPPR', '5', '7', 'U', 'ENTER', 'CTRL', 'L', 'H', 'S', 'SUPPR', 'ENTER', 'S', 'CTRL', 'SHIFT', 'I', 'T', 'O', 'M', 'V', 'SUPPR', 'I', 'J', '0', 'M', 'I', 'M', 'Q', 'Y', '5', 'L', 'N', 'R', '9', 'CTRL', 'CTRL', '8', 'H', 'K', 'W', 'Y', 'CTRL', 'SPACE', 'H', 'H', '6', 'L', 'W', '0', 'K', 'O', '6', '6', 'G', 'A', 'L', 'SUPPR', 'Z', 'H', 'R', 'S', 'M', 'S', 'A', 'E', 'SUPPR', '0', '6', 'D', '0', 'Z', 'Q', '9', 'X', 'NOKEY', '6', 'SPACE', 'SPACE', 'SPACE', 'U', 'F', 'D', 'Y', '8', 'ENTER', 'R', 'Q', '5', '7', 'W', 'W', 'V', '8', 'N', 'Y', 'J', '1', '8', 'S', 'T', '8', 'G', 'N', '7', 'ENTER', 'Y', 'B', 'NOKEY', '2', 'SHIFT', 'J', 'T', 'D', 'ENTER', 'M', 'U', 'J', 'F', 'B', '2', '5'

In [221]:
""" Code poubelle -> tests
pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")
values, info = get_pics_from_file("../data/pics_3.bin")
print(np.mean(values,axis=0))
np.random.shuffle(values)
plt.imshow(values[:3000],aspect="auto")
plt.show()
nokey = np.mean(pics_nokey,axis=0)
denoised = np.subtract(values[:3000],nokey)
plt.imshow(denoised,aspect="auto")
plt.show()
mean = np.mean(values[:3000],axis=0)
cleaned = values[:3000] * mean
plt.imshow(cleaned,aspect="auto")
plt.show()

denoised = np.subtract(cleaned,nokey)
plt.imshow(denoised,aspect="auto")
plt.show()
"""

' Code poubelle -> tests\npics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")\nvalues, info = get_pics_from_file("../data/pics_3.bin")\nprint(np.mean(values,axis=0))\nnp.random.shuffle(values)\nplt.imshow(values[:3000],aspect="auto")\nplt.show()\nnokey = np.mean(pics_nokey,axis=0)\ndenoised = np.subtract(values[:3000],nokey)\nplt.imshow(denoised,aspect="auto")\nplt.show()\nmean = np.mean(values[:3000],axis=0)\ncleaned = values[:3000] * mean\nplt.imshow(cleaned,aspect="auto")\nplt.show()\n\ndenoised = np.subtract(cleaned,nokey)\nplt.imshow(denoised,aspect="auto")\nplt.show()\n'

In [82]:
guess, info = get_pics_from_file("../data/pics_LOGINMDP.bin")
output = []

"""def flooring(a):
    if a > 2.4:
        return 1
    return 0"""

guess = np.subtract(guess, nokey)
chunks = []
chunks_index = []
empty = np.subtract(nokey, nokey)
i = 0
threshold = 1.4
min_nokey_length = 10
while i < len(guess):
    nokey_length = 0
    while i < len(guess) and np.linalg.norm(guess[i] - empty) < threshold:
        nokey_length += 1
        i += 1
    if nokey_length > min_nokey_length:
        chunks.append([])
        chunks_index.append(i)
    while i < len(guess) and np.linalg.norm(guess[i] - empty) >= threshold:
        if len(chunks) == 0:
            chunks.append([])
            chunks_index.append(i)
        chunks[-1].append(guess[i])
        i += 1
j = 0
print("cleaning noise")
while j < len(chunks):
    if len(chunks[j]) < 5:
        chunks.pop(j)
        chunks_index.pop(j)
    else:
        j += 1
"""for index, frame in enumerate(guess):
    if np.linalg.norm(last - frame) < 1.5:
        chunks[-1].append(frame)
    else:
        chunks.append([frame])
        chunks_index.append(index)
        last = frame"""
print(len(chunks_index))
print(chunks_index)
# for chunk in chunks:
#     print(len(chunk))


cleaning noise
14
[1821, 3014, 6018, 6328, 6617, 7021, 7495, 7879, 8178, 8547, 9251, 9861, 10213, 10235]


In [81]:
def copy_curr(array):
    result = []
    for np_array in array:
        result.append(np.copy(np_array))
    return result

for current in chunks:
    curr = np.array(current)
    if len(curr) < 20:
        continue
    print(len(curr))
    #curr = np.subtract(curr,nokey)
    mean = np.mean(curr,axis=0)
    curr = curr * mean
    curr = sc_X.transform(curr)
    curr_output = classifier.predict(curr)
    unique,pos = np.unique(curr_output,return_inverse=True)
    max = unique[np.bincount(pos).argmax()]
    if max == 'NOKEY':
        print("nokey")
        continue
    if max == 'SHIFT':
        curr = np.array(current)
        #erase pic 5 (shift)
        for j,frame in enumerate(curr):
            frame[5] = 0
            curr[j] = frame
        #curr = np.subtract(curr,nokey)
        mean = np.mean(curr,axis=0)
        curr = curr * mean
        curr = sc_X.transform(curr)
        curr_output = classifier.predict(curr)
        unique,pos = np.unique(curr_output,return_inverse=True)
        max = unique[np.bincount(pos).argmax()]
        if max == 'NOKEY' or max == 'SHIFT':
            print("nokey or shift only")
            continue
        if output == [] or ("SHIFT " + max) != output[-1]:
            output.append("SHIFT " + max)
    elif output == [] or max != output[-1]:
        output.append(max)

#probs = classifier.predict_proba(input)
#best_n = np.argsort(-probs, axis=1)[:, :3]
#print(best_n)


print("len: ",len(output))
for a in output:
    print(a)


301
1365
69
24
80
65
54
len:  7
CTRL
SHIFT M
C
M
O
N
ENTER
